In [64]:
import os
import json
import pandas as pd

In [65]:
with open('../configs/output.manifest','r') as f:
    dataset = f.read().split('\n')

In [66]:
def format_annotation(img_h, img_w, x_top, y_top, box_w, box_h, class_id):
    center_x = round((x_top + (box_w / 2)) / img_w,4)
    center_y = round((y_top + (box_h / 2)) / img_h,4)
    w = round(box_w / img_w,4)
    h = round(box_h / img_h,4)
    anno_dict = {
        'label' : class_id,
        'center_x' : center_x,
        'center_y' : center_y,
        'w' : w,
        'h' : h
    }
    return anno_dict
                

def get_annotations(anno_info, label_info):
    img_h = anno_info.get('image_size')[0].get('height')
    img_w = anno_info.get('image_size')[0].get('width')
    temp_anno_list = []
    for anno in anno_info.get('annotations'):
        class_id = anno.get('class_id')
        x_top = anno.get('left')
        y_top = anno.get('top')
        box_w = anno.get('width')
        box_h = anno.get('height')
        anno_dict = format_annotation(img_h, img_w, x_top, y_top, box_w, box_h, class_id)
        temp_anno_list.append(anno_dict)
    return temp_anno_list
        
    

In [67]:
image_path_list = []
image_name_list = []
anno_list = []
is_background_list = []


for image_data in dataset[:-1]:
    image_data = json.loads(image_data)
    image_path = image_data.get('source-ref')
    anno_info = image_data.get('category')
    label_info = image_data.get('category-metadata').get('class-map')
    image_anno_list = get_annotations(anno_info, label_info)
    
    image_path_list.append(image_path)
    image_name_list.append(os.path.basename(image_path))
    anno_list.append(image_anno_list)
    if len(image_anno_list) == 0:
        is_background_list.append(True)
    else:
        is_background_list.append(False)

In [68]:
data_df = pd.DataFrame({
    "image_name" : image_name_list,
    "s3_path" : image_path_list,
    "annotations" : anno_list,
    "is_background" : is_background_list
    }
)

In [69]:
data_df.head()

,image_name,s3_path,annotations,is_background
0,maksssksksss0.png,s3://sixsense-organization-assets/ayush/labeli...,"[{'label': 0, 'center_x': 0.4092, 'center_y': ...",False
1,maksssksksss1.png,s3://sixsense-organization-assets/ayush/labeli...,"[{'label': 0, 'center_x': 0.61, 'center_y': 0....",False
2,maksssksksss2.png,s3://sixsense-organization-assets/ayush/labeli...,"[{'label': 0, 'center_x': 0.2213, 'center_y': ...",False
3,maksssksksss3.png,s3://sixsense-organization-assets/ayush/labeli...,"[{'label': 0, 'center_x': 0.4963, 'center_y': ...",False
4,maksssksksss4.png,s3://sixsense-organization-assets/ayush/labeli...,"[{'label': 0, 'center_x': 0.4252, 'center_y': ...",False


In [70]:
data_df.annotations.tolist()[0]

[{'label': 0,
  'center_x': 0.4092,
  'center_y': 0.3538,
  'w': 0.1309,
  'h': 0.1339}]

In [75]:
def prepare_label_files(df, label_save_path):
    for idx,row in df.iterrows():
        if row['is_background']:
            continue
        image_path = row['s3_path']
        image_name = os.path.basename(image_path)
        image_extension = image_name.split('.')[-1]
        label_file_name = image_name.replace(f".{image_extension}",".txt")
        print(image_name, image_extension, label_file_name)
        with open(os.path.join(label_save_path,label_file_name),'w') as f:
            for anno in row['annotations']:
                f.writelines(f"{anno['label']} {anno['center_x']} {anno['center_y']} {anno['w']} {anno['h']}\n")

In [76]:
save_path = '../data/sample/'
os.makedirs(save_path,exist_ok=True)
prepare_label_files(data_df,save_path)

maksssksksss0.png png maksssksksss0.txt
maksssksksss1.png png maksssksksss1.txt
maksssksksss2.png png maksssksksss2.txt
maksssksksss3.png png maksssksksss3.txt
maksssksksss4.png png maksssksksss4.txt
maksssksksss5.png png maksssksksss5.txt
maksssksksss6.png png maksssksksss6.txt


In [79]:
import boto3
resource = boto3.resource('s3')
my_bucket = resource.Bucket('sixsense-organization-assets')
my_bucket.download_file('ayush/labeling_job_test/dataset-small/maksssksksss4.png','./maksssksksss4.png')

In [81]:
from ultralytics import YOLO
import warnings 
warnings.filterwarnings('ignore')
model = YOLO('../runs/detect/train/weights/best.pt')

In [105]:
img = '../data/test1/images/val/maksssksksss6.png'
params = {'conf' : 0.01}
result = model.predict(img, **params)


image 1/1 /Users/dare_devil/Documents/pythonProject/Asurion/AWS_SAGEMAKER_PIPELINE/training/data/test1/images/val/maksssksksss6.png: 448x640 82 MASKs, 79.2ms
Speed: 0.9ms pre-process, 79.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


In [108]:
for r in result:
    print(r.boxes.xywhn)
    print(r.boxes.cls)

tensor([[0.48917, 0.50988, 0.71167, 0.96543],
        [0.90833, 0.70864, 0.18333, 0.37037],
        [0.50000, 0.73951, 0.68000, 0.50123],
        [0.05833, 0.80494, 0.11667, 0.37531],
        [0.77667, 0.73086, 0.44667, 0.53333],
        [0.78083, 0.69136, 0.20500, 0.50864],
        [0.84083, 0.74938, 0.31833, 0.49630],
        [0.14167, 0.79753, 0.28333, 0.38519],
        [0.05667, 0.67654, 0.11333, 0.63210],
        [0.19250, 0.76420, 0.38167, 0.42716],
        [0.51417, 0.87407, 0.64833, 0.23704],
        [0.49000, 0.30988, 0.35333, 0.57037],
        [0.03583, 0.55062, 0.07167, 0.83457],
        [0.16667, 0.23827, 0.33333, 0.47654],
        [0.20667, 0.09506, 0.41333, 0.19012],
        [0.07417, 0.24691, 0.14833, 0.49383],
        [0.86333, 0.23457, 0.27333, 0.46914],
        [0.66667, 0.30000, 0.63667, 0.60000],
        [0.02583, 0.40617, 0.05167, 0.59012],
        [0.11917, 0.13210, 0.23833, 0.26420],
        [0.86167, 0.59383, 0.27667, 0.70864],
        [0.72083, 0.93210, 0.55833